In [10]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

In [11]:
#load all of our documents
with open('NHlegtext09_20.corpus','rb') as f:
    legcorpus = pickle.load(f)
    
#print a random doc
print(legcorpus[303])

#assemble the text data, and remove the part where it says if it passes or not
combined_docs = {} #{bill_id:text}
total = 0
passed = 0

#a more unorthodox method of grabbing the target passage
leg_target = {} #{bill: 0 or 1, where 1 means pass}

#for each document, check if it is approved, record that, then combine the list into a string
for doc in legcorpus:
    pass_flag = 0
    if doc[0] not in combined_docs:
        total += 1
        try:
            if doc[2][-5] == 'approved':
                doc[2] = doc[2][:-5]
                passed += 1
                pass_flag = 1
                combined_docs[doc[0]] = " ".join(doc[2])
            else:
                combined_docs[doc[0]] = " ".join(doc[2])
                
        except:
            combined_docs[doc[0]] = " ".join(doc[2])
            pass
                
    leg_target[doc[0]] = pass_flag
    #for now, ignoring the amendments, but it seems like it could do better with the amendments
    #that would go here (i.e. an else statement saying doc[0] in combined_docs)
    
                   
print("\nTotal bills:", total)
print("Passed bills:", passed)
print("Bill passed ratio:", passed/total)

[574238, 1018561, ['act', 'establishing', 'permanent', 'subcommittee', 'health', 'human', 'services', 'oversight', 'committee', 'relative', 'alzheimer', 'disease', 'related', 'dementia', 'requiring', 'certain', 'training', 'education', 'programs', 'regarding', 'alzheimer', 'disease', 'related', 'dementia', 'enacted', 'senate', 'house', 'representatives', 'general', 'court', 'convened', 'new', 'section', 'health', 'human', 'services', 'oversight', 'committee', 'permanent', 'subcommittee', 'alzheimer', 'disease', 'related', 'dementia', 'amend', 'rsa', 'inserting', 'section', 'following', 'new', 'section', 'subcommittee', 'alzheimer', 'disease', 'related', 'dementia', 'addition', 'subcommittees', 'established', 'rsa', 'ii', 'health', 'human', 'services', 'oversight', 'committee', 'shall', 'create', 'permanent', 'subcommittee', 'alzheimer', 'disease', 'related', 'dementia', 'ii', 'members', 'subcommittee', 'shall', 'follows', 'one', 'member', 'house', 'representatives', 'appointed', 'speak

In [12]:
#actually, lets just stick with the given passages for now
leg_data = pd.read_csv("NHLegOverallData09_20.csv") #need all 10000, this is just 3000


with open('NHStateTargetList09_20.list','rb') as f:
    event_target = pickle.load(f)

with open('NHWinList09_20.list','rb') as f:
    win_state = pickle.load(f)
    
leg_targ = pd.DataFrame({"every stage passed":win_state}) #need all 10000, this is just 3000
##################check above for correct inputs





bill_order = []
bill_target = []

leg_targ_old = leg_target
leg_target = {}

for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
        
           
# bill_texts = []
# for bill_id in bill_order:
#     if bill_id in combined_docs:
#         bill_texts.append(combined_docs[bill_id])
#     else:
#         bill_texts.append("")
passed = 0
total = 0
for i in leg_target:
    passed+=leg_target[i]
    total += 1
print(passed/total)
#print(leg_target)

0.32020354315868826


In [13]:
print(passed)
print(total)

3398
10612


In [8]:
#features are in a dictionary. 
with open('BillFeatures.dict','rb') as f:
    features = pickle.load(f)
    
X = pd.DataFrame(features)
#print(X['committee introduced'].head())

#make a mask so we only keep the rows that we actually have text for
X_mask = [False]*len(X)
for i, bill_id in enumerate(combined_docs): 
    try:
        print(X['bill_id'][X['bill_id'] == bill_id].index)
        X['bill_id'][X['bill_id'] == bill_id].index[0]
        X_mask[X['bill_id'][X['bill_id'] == bill_id].index[0]] = True
    except:
        print("i:",i,"bill_id:",bill_id)
        raise IndexError 

#transform the array of committee introduced into a string
committees = []
for committee in X['committee introduced']:
    committees.append(" ".join(committee))
X['committee introduced'] = committees

#next step: sift X so that it only contains ones we have docs for
X_mask = pd.Series(X_mask)
print(X_mask.iloc[4:9])
X = X[X_mask]

X = X.astype({'bill_id': 'int64'}).sort_values('bill_id', ascending=True, ignore_index=True)
X = X.astype({'year introduced': 'int64'}).sort_values('year introduced', ascending=True, ignore_index=True)
#date = #make date introduced something like 2018.50, so that we can sort numerically by month too
X = X.set_index('bill_id')

print(X.loc[894687]['title'])
X.head()

Int64Index([], dtype='int64')
i: 0 bill_id: 576685


IndexError: 

In [ ]:
#align the documents with their row to make one complete dataset
ordered_docs = [""]*len(combined_docs)

#align the targets with their row
ordered_target = [0]*len(combined_docs)
#print(leg_target)

#take the bill ids from the dataframe (which are in order)
ordered_bill_ids = list(X.index.values)
for i, b_id in enumerate(ordered_bill_ids):
    ordered_docs[i] = combined_docs[str(b_id)]
    ordered_target[i] = leg_target[str(b_id)]
    if str(b_id) == '894761':
        print(i)
        print(ordered_target[i])
    
#make a new column for text    
X['legtext'] = ordered_docs


In [ ]:
#select the data that we actually want to train on
#it is probably better to use primary id in the future instead of primary name
X_new = X[['house/sentate', 'primary name', 'primary party', 'number of co sponsors', 'committee introduced','month introduced']].copy() #, 'committee introduced'
print(ordered_target[0:5])

amount_train = 1970
amount_test = 500

X_dum = pd.get_dummies(X_new)

#can either manually define the features, or uncomment the below feature selection
X_dum = X_dum[['number of co sponsors', 'house/sentate_S', 'month introduced_01', 'month introduced_05', 'month introduced_06', 'month introduced_12', 'primary party_R']]

X_train = X_dum.iloc[0:amount_train]#.iloc
y_train = ordered_target[0:amount_train]
X_test = X_dum.iloc[amount_train:amount_train+amount_test]#.iloc
y_test = ordered_target[amount_train:amount_train+amount_test]

# indices = SelectKBest(f_classif, k=15).fit(X_train, y_train).get_support(indices=True)
# print(X_train.iloc[0,indices])
# #print('number of co sponsors')
# k_select = SelectKBest(f_classif, k=15)
# X_train = k_select.fit_transform(X_train, y_train)
# X_test = k_select.transform(X_test)
# #indices = 

# print(sum(y_train)/len(y_train))
# print(sum(y_test)/len(y_test))
#print(X_train[0:10])
#print(X_test[0:10])
X_train.head()

In [ ]:
#prepare the documents by using tfidf vectors:

#from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#adjust this stemmer so that it does more than one word in the tokens

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,4))
stem_counts = stemmed_count_vect.fit_transform(X['legtext'][0:amount_train])
stem_counts_test = stemmed_count_vect.transform(X['legtext'][amount_train:amount_train+amount_test])

tfidf = TfidfTransformer()
tfidf_vectors = tfidf.fit_transform(stem_counts)
tfidf_vectors_test = tfidf.transform(stem_counts_test)

k_best_tfidf = SelectKBest(f_classif, k=1000)
final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, y_train)
final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)

In [ ]:
#combine tfidf legtext and the other features:
legtext_train = final_tfidf_features.todense()
#legtext_train = pd.DataFrame(legtext_train)#*10

legtext_test = final_tfidf_features_test.todense()
#legtext_test = pd.DataFrame(legtext_test)


X_train = np.concatenate([np.array(X_train),legtext_train], axis=1)
X_test = np.concatenate([np.array(X_test),legtext_test], axis=1)
#X_train.head()
X_train[0]

In [ ]:
#train the neural network a bunch
means = []
for i in range(20):
    clf = MultinomialNB()#MLPClassifier(alpha=0.0001, random_state=i, max_iter=1000)#19 ##
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    predicted_prob = clf.predict_proba(X_test)
    train_prediction = clf.predict(X_train)
    new_mean = np.mean(predicted == y_test)
    means.append(new_mean)
    print(new_mean)
    #print(np.mean(train_prediction == y_train))
    
mean=np.mean(np.array(means))
print("overall mean:", mean)

In [ ]:
print(mean)

In [ ]:

list(X_dum.keys()[indices])


In [ ]:
#only run this if using MultinomialNB classifier
feature_indices = indices
pass_probs = clf.feature_log_prob_[1]
feature_pass_probs = {feature:pass_probs[i] for i, feature in enumerate(feature_indices)}

sorted_features = sorted(feature_indices, key=lambda x: feature_pass_probs[x], reverse=True)
#print(sorted_features)

vocab_dict = {indices[i]:X_dum.keys()[indices][i] for i, _ in enumerate(indices)}
#print(vocab_dict)

for i, feature in enumerate(sorted_features):
    if i >= 1000:
        break
    print(vocab_dict[feature], "\t\t", feature_pass_probs[feature])

In [ ]:
print(len(predicted))
print()
for i, prediction in enumerate(predicted):
    if i == 50:
        break
    print("real:", y_test[i],"\tpredicted:", predicted_prob[i][1])#, "\tdata:", X_dum.iloc[amount_train:amount_train+amount_test,indices])
